In [1]:
from __future__ import absolute_import, division, unicode_literals
import sys
import io
import numpy as np
import logging
import argparse
import torch
import random
from transformers import *
import utils
import json
import pandas as pd
from tqdm import tqdm


/root/anaconda3/envs/SBERT-WK/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sbert import sbert

2022-05-13 01:04:48,891 : Starting new HTTPS connection (1): s3.amazonaws.com:443
2022-05-13 01:04:49,930 : https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/binwang/bert-base-nli-stsb/config.json HTTP/1.1" 200 0
2022-05-13 01:04:49,932 : loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/binwang/bert-base-nli-stsb/config.json from cache at ./cache/da86f44c4219e194281dc8357b2f3b710998009231581665837f1dc21b08fb75.2f62e73f3306183a4b5f94267e4695f8afd4c3f2b29a4c4df18f8fecd046e609
2022-05-13 01:04:49,933 : Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "out

to device


In [28]:
sbert(['A bear chasing a bicyclist down a mountain road',
'Photograph of a bear chasing a cyclist'])

0.84155595

In [29]:
sbert(['A bear chasing a bicyclist down a mountain road is fake',
'Photograph of a bear chasing a cyclist'])

0.7704923

In [30]:
sbert(['A bear chasing a bicyclist down a mountain road',
'Photograph of a bear chasing a cyclist is fake'])

0.6986189

In [3]:
import itertools
# Get max sim between 1 sentence and others
def get_max_sim(sentence, sentences):
    max_sim = 0
    for pair in itertools.product(sentence,sentences):
        sim = sbert([pair[0],pair[1]])
        if sim > max_sim:
            max_sim = sim
    return max_sim

In [4]:
import itertools
# Get max sim between 1 sentence and others
def get_mean_sim(sentence, sentences):
    mean_sim = []
    for pair in itertools.product(sentence,sentences):
        sim = sbert([pair[0],pair[1]])
        mean_sim.append(sim)
    return np.mean(mean_sim)

In [20]:
# df = pd.read_json('/root/thesis/ViLT/cosmos/test_data.json',orient="records", lines=True)

In [5]:
df = pd.read_csv('new_data.csv')

In [6]:
tqdm.pandas()
df['sbert_wk_score_m'] = df.progress_apply(lambda x: sbert([x.caption1_modified,x.caption2_modified]), axis=1)

100%|██████████| 1700/1700 [00:26<00:00, 65.29it/s]


In [7]:
df['sbert_wk_score'] = df['bert_base_score']

In [6]:
df['caption_3'] = df['caption_3'].apply( lambda x: x.strip("[]").replace("'","").split(", "))

In [8]:
tqdm.pandas()
df['sbert_wk_score_1_3'] = df.progress_apply(lambda x: 
get_mean_sim([x.caption1_modified],x.caption_3)
, axis=1)
df['sbert_wk_score_2_3'] = df.progress_apply(lambda x: 
get_mean_sim([x.caption2_modified],x.caption_3)
, axis=1)

  2%|▏         | 27/1700 [01:01<1:02:46,  2.25s/it]

In [8]:
def get_false_scores(row):
    new_scores = [0,0,0,0]
    c1_fake = row['caption1'][:-1] + ' is not genuine.'
    new_scores[0] = sbert([c1_fake,row['caption2']])

    c1_fake = row['caption1'][:-1] + ' is fake.'
    new_scores[1] = sbert([c1_fake,row['caption2']])

    c1_fake = row['caption1'][:-1] + ' wasn\'t true.'
    new_scores[2] = sbert([c1_fake,row['caption2']])

    c1_fake = 'No, ' + row['caption1'][:-1]
    new_scores[3] = sbert([c1_fake,row['caption2']])
    return new_scores
df['false_scores'] = df.progress_apply(lambda x: get_false_scores(x), axis=1)

100%|██████████| 1700/1700 [02:12<00:00, 12.84it/s]


In [9]:
df['bb']= pd.read_csv('/root/thesis/COSMOS/pred_contexts.txt', header=None)[0]

In [10]:
def predict(row):
    false_scores = row['false_scores']
    if false_scores[0] > row['sbert_wk_score'] + 0.05:
        return True
    if false_scores[1] > row['sbert_wk_score'] + 0.05:
        return True
    if false_scores[2] > row['sbert_wk_score'] + 0.05:
        return True
    if false_scores[3] > row['sbert_wk_score'] + 0.05:
        return True

    if row['sbert_wk_score']>0.5:
        if row['sbert_wk_score']>0.7:
            if false_scores[0] > row['sbert_wk_score'] + 0.02:
                return True
            if false_scores[1] > row['sbert_wk_score'] + 0.02:
                return True
            if false_scores[2] > row['sbert_wk_score'] + 0.02:
                return True
            if false_scores[3] > row['sbert_wk_score'] + 0.02:
                return True

        # 52 cau similar, khong doi lap nhung OOC
        # if row['context_label']==1:
        #     print('g')
        return False
    
    if row['sbert_wk_score_1_3'] > 0.5 or row['sbert_wk_score_2_3'] > 0.5 :
        # 18 cau khong doi lap, captiong match nhung OOC
        return False
    # 26 cau sai nhung no thay dung
    # 75 cau dung nhung no thay sai
    # Tong cau o day la 569. 17% cau o day sai
    return row['bb']==True
    # if row['iou']<0.5:
    #     return False

    return True

In [42]:
# Language Baseline
df['predict'] =  (df['sbert_wk_score'] < 0.5)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      591  134
True       259  716
accuracy: 0.7688235294117647


In [14]:
# Language Baseline + neu phu dinh
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      571   52
True       279  798
accuracy: 0.8052941176470588


In [16]:
# Language Baseline + neu phu dinh + image_caption threshold 0.5
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)

Actual       0    1
Predicted          
False      621   75
True       229  775
accuracy: 0.8211764705882353


In [11]:
# Language Baseline + neu phu dinh + image_caption + cosmos
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      722  101
True       128  749
accuracy: 0.8652941176470588


In [12]:
# Language Baseline + neu phu dinh + image_caption + iou
df['predict'] =  df.apply(lambda x:predict(x), axis=1)
confusion_matrix = pd.crosstab(df['predict'], df['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
print('accuracy:', result)
#0.808

Actual       0    1
Predicted          
False      722  101
True       128  749
accuracy: 0.8652941176470588


In [14]:
col=['caption1','caption2','context_label','sbert_wk_score',
    'predict','caption_3',
    'sbert_wk_score_1_3','sbert_wk_score_2_3'
    ]
df[col].to_csv('86_52.csv',index=False)

In [83]:
df.loc[312]

img_local_path                                             test/312.jpg
caption1              Office worker Hideyuki Togashi, 48, shows his ...
caption2              A man stands on grass wearing shorts, showing ...
context_label                                                         0
article_url               https://www.bbc.com/news/in-pictures-54903538
maskrcnn_bboxes       [[304.6567077636719, 9.236845016479492, 637.36...
caption1_modified     Office worker PERSON, DATE, shows his tattoos ...
caption1_entities     [['Hideyuki Togashi', 'PERSON'], ['48', 'DATE'...
caption2_modified     A man stands on grass wearing shorts, showing ...
caption2_entities                                                    []
bert_base_score                                                0.090015
bert_large_score                                               0.206176
nouns_1               ['Office worker PERSON', 'DATE', 'his tattoos'...
nouns_2               ['A man', 'grass', 'shorts', 'a prosthetic

In [169]:
sbert(['A young girl disconnected her grandfather\'s life support in order to charge her mobile phone is fake.',
'A young girl disconnected her grandfather\'s life support in order to charge her mobile phone is purely a work of fiction.'])

0.8266825

In [165]:
df[(df['predict']==0)&(df['context_label']==1)][col].sample(3)

,caption1,caption2,context_label,sbert_wk_score,predict,method,caption_3,sbert_wk_score_1_3,sbert_wk_score_2_3
1302,A young girl disconnected her grandfather's life support in order to charge her mobile phone.,A young girl disconnected her grandfather's life support in order to charge her mobile phone is purely a work of fiction.,1,0.799659,False,None,"[a person holding a cell phone on top of a table, person holding a cell phone with a its on the table, person holding a cell phone in front of this table]",0.169527,0.110222
150,A photograph shows U.S. President Joe Biden on a movie set designed to look like the Oval Office.,"This photograph was taken on Feb. 5, 2021, as Biden and Vice President Kamala Harris held a meeting in the actual Oval Office to discuss their COVID-19 relief package.",1,0.349611,False,None,"[a group of people sitting in a living room, a group of people sitting in a living room, a group of people sitting in this living room]",0.346654,0.248080
767,"A photograph shows a group of buses that were used to ship in paid anti-Trump protesters to Austin, Texas.",Photographs showing long lines of buses were shared with the untrue claim that they were used to ship paid anti-Trump protesters to various cities.,1,0.748869,False,None,"[this is a group of people holding signs in the street, a group of people holding signs and on the street, a group of people walking down the street with signs]",0.220911,0.186781


In [73]:
sbert(['Members of the migrant caravan dragging a Mexican official through the streets','a group of people riding skateboards down the street'])

0.3423701

In [91]:
pd.set_option('display.max_colwidth', None)
df[(df['sbert_wk_score']<0.5)&(df['context_label']==0)][col].loc[48]

caption1                                                                                                             The Cow and the Tailor Shop, Rajasthan, by Elizabeth Brooks, from the US
caption2                                                                                                                                                    A sacred cow in a street in India
context_label                                                                                                                                                                               0
sbert_wk_score                                                                                                                                                                        0.48704
predict                                                                                                                                                                                  True
method                                            

In [92]:
vilt = pd.read_csv('/root/thesis/ViLT/vilt.csv')

In [94]:
df['vilt'] = vilt['ooc_score']

In [70]:
df[(df['predict']==1) & (df['context_label']!=df['predict'])][['caption1','caption2','context_label','predict']]

,caption1,caption2,context_label,predict
9,A man brushes the mouth of a sarcophagus.,More than 40 statues of ancient deities and fu...,0,True
11,"In Bangkok, tuk-tuks that used to transport to...",Tuk-tuk car parked in Bangkok because there ar...,0,True
15,Thunberg already has several murals around the...,San Francisco's latest mural features famous c...,0,True
16,"On the same day in South Africa, a bird walks ...",General view of an empty beach with high-rise ...,0,True
20,"Martin Fayulu, the opposition leader and runne...",Opposition presidential candidate in the Democ...,0,True
...,...,...,...,...
1678,A photographer's self portrait with her mother...,"Two women embrace, wearing face masks",0,True
1679,Buildings around the UK were also lit in blue ...,"Falkirk is home to The Kelpies, the largest eq...",0,True
1687,Oggi Tomic taking a photo,A child holds a camera next to a destroyed bus,0,True
1691,A man exercises on a street in Edinburgh,A man jumps over a skipping rope in a street,0,True


In [44]:
z=['test/335.jpg',
'test/586.png',
'test/909.jpg',
'test/971.png',
'test/1029.jpg',
'test/1077.png']
df[df['img_local_path'].isin(z)]

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,...,nouns_1,nouns_2,caption_3,bbs,bbs2,iou,sbert_wk_score,sbert_wk_score_2,predict,fl
335,test/335.jpg,A photograph shows a purple lobster caught in ...,One-in-a-million' purple lobster fools the int...,1,https://www.snopes.com/fact-check/purple-lobst...,"[[22.697582244873047, 0.0, 832.2989501953125, ...",A photograph shows a purple lobster caught in ...,"[['Maine', 'GPE']]",CARDINAL-in-a-million' purple lobster fools th...,"[['One', 'CARDINAL']]",...,"['A photograph', 'a purple lobster', 'GPE']","['CARDINAL-in-a-million purple lobster', 'the ...",[there is purple lobsters sitting in the water...,"[[0.3705366551876068, 0.47459208965301514, 0.3...","[[0.4393530488014221, 0.44597700238227844, 0.4...",0.720439,0.429207,0.813934,True,True
586,test/586.png,A bear chasing a bicyclist down a mountain road,This is not a genuine photograph of a bear cha...,1,https://www.snopes.com/fact-check/bear-chases-...,"[[146.42872619628906, 383.7769775390625, 233.6...",A bear chasing a bicyclist down a mountain road,[],This is not a genuine photograph of a bear cha...,[],...,"['A bear', 'a bicyclist', 'a mountain road']","['This', 'a genuine photograph', 'a bear', 'a ...","[a man riding a bike down the side of a road, ...","[[0.4303147792816162, 0.5148906111717224, 0.21...","[[0.4308185577392578, 0.44097715616226196, 0.5...",0.535473,0.666386,0.742175,True,True
909,test/909.jpg,A juvenile male humpback whale in waters off S...,Scientists listening to whale songs suspect th...,1,https://www.nytimes.com/2020/12/23/science/blu...,"[[119.96807098388672, 233.91754150390625, 280....",A juvenile male humpback whale in waters off GPE.,"[['Sri Lanka', 'GPE']]",Scientists listening to whale songs suspect th...,[],...,"['A juvenile male humpback whale', 'waters', '...","['Scientists', 'whale songs', 'other reasons',...",[an elegant carm the watering over its belly i...,"[[0.4776133596897125, 0.462051659822464, 0.755...","[[0.6149105429649353, 0.5887476205825806, 0.50...",0.528499,0.276388,0.667675,True,True
971,test/971.png,A bear chasing a bicyclist down a mountain road,The original photograph (which featured a bear...,1,https://www.snopes.com/fact-check/bear-chases-...,"[[146.42872619628906, 383.7769775390625, 233.6...",A bear chasing a bicyclist down a mountain road,[],The original photograph (which featured a bear...,"[['September 2014', 'DATE'], ['Adonis Arias’',...",...,"['A bear', 'a bicyclist', 'a mountain road']","['The original photograph', 'which', 'a bear',...","[a man riding a bike down the side of a road, ...","[[0.2525228261947632, 0.5491107106208801, 0.27...","[[0.4918614327907562, 0.4739872217178345, 0.59...",0.354941,0.311818,0.716651,True,True
1029,test/1029.jpg,Oliver Rattray kicks off the latest gallery wi...,Ice cream van in fog at Glasgow's Queen's Park.,0,https://www.bbc.com/news/uk-scotland-55546350,"[[650.4645385742188, 290.1622009277344, 714.01...",PERSON kicks off the latest gallery with this ...,"[['Oliver Rattray', 'PERSON'], ['Glasgow', 'GP...",Ice cream van in fog at GPE's FAC.,"[['Glasgow', 'GPE'], [""Queen's Park"", 'FAC']]",...,"['PERSON', 'the latest gallery', 'this ice cre...","['Ice cream van', 'fog', 'ORG']",[a group of people standing in front of an ice...,"[[0.4815254211425781, 0.5841901898384094, 0.13...","[[0.485992968082428, 0.5401358604431152, 0.540...",0.475667,0.588501,0.621402,True,True
1077,test/1077.png,A photograph captures mounds of refuse and oth...,The appearance of the area “was caused by a li...,1,https://www.snopes.com/fact-check/seattle-chaz...,"[[595.9969482421875, 44.270931243896484, 639.1...",A photograph captures mounds of refuse and oth...,"[['Seattle', 'GPE'], ['Capitol Hill Autonomous...",The appearance of the area “was caused by a li...,[],...,"['A photograph', 'mounds', 'refuse', 'other tr...","['The appearance', 'the area', 'a lit cigarett...","

In [333]:
df[df['predict']!=df['context_label']].sample()

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,...,bert_large_score,nouns_1,nouns_2,caption_3,bbs,bbs2,iou,sbert_wk_score_2,sbert_wk_score,predict
774,test/774.jpg,A girl stands in the flooded courtyard of a sc...,A girl stands on a piece of wood in the floode...,0,https://www.bbc.com/news/in-pictures-55846176,"[[463.83343505859375, 185.97604370117188, 602....",A girl stands in the flooded courtyard of a sc...,"[['Beira', 'GPE'], ['Mozambique', 'GPE'], ['Af...",A girl stands on a piece of wood in the floode...,"[['Beira', 'GPE'], ['Mozambique', 'GPE'], ['27...",...,0.49228,"['A girl', 'the flooded courtyard', 'a school'...","['A girl', 'a piece', 'wood', 'the flooded cou...",[this girl standing in the water on an almost ...,"[[0.442098468542099, 0.5239864587783813, 0.194...","[[0.45846012234687805, 0.5268453359603882, 0.2...",0.908733,0.407105,0.477712,True


In [335]:
df[(df['predict']==True) & (df['context_label']==False)].sample(5)

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,...,bert_large_score,nouns_1,nouns_2,caption_3,bbs,bbs2,iou,sbert_wk_score_2,sbert_wk_score,predict
555,test/555.jpg,"it's a runway with a difference, beneath the b...",A fashion model walks down the runway during D...,0,https://www.bbc.com/news/world-africa-55346788,"[[309.20367431640625, 239.4454345703125, 379.6...","it's a runway with a difference, beneath the b...","[['Senegal', 'GPE']]",A fashion model walks down the runway during O...,"[['Dakar Fashion Week', 'ORG'], ['Dakar', 'GPE...",...,0.153872,"['it', 'a runway', 'a difference', 'the baobab...","['A fashion model', 'the runway', 'EVENT', 'PE...",[a group of people walking down the side of a ...,"[[0.5190181136131287, 0.5266764760017395, 0.59...","[[0.5678038597106934, 0.5720387697219849, 0.14...",0.487122,0.303624,0.118213,True
690,test/690.jpg,"Untitled 01, Coney Island, by Hugo de Melo, fr...",Two women are seen out shopping in the street,0,https://www.bbc.com/news/in-pictures-53234985,"[[630.728759765625, 98.6261978149414, 886.0698...","Untitled CARDINAL, LOC, by EVENT, from the GPE","[['01', 'CARDINAL'], ['Coney Island', 'LOC'], ...",CARDINAL women are seen out shopping in the st...,"[['Two', 'CARDINAL']]",...,-0.087656,"['Untitled CARDINAL', 'GPE', 'PERSON', 'the GPE']","['CARDINAL women', 'the street']",[two women walking down the street talking on ...,"[[0.3545509874820709, 0.22674578428268433, 0.3...","[[0.5390362739562988, 0.5792112946510315, 0.77...",0.326540,0.480872,-0.098430,True
1090,test/1090.jpg,"On Wednesday, a student protester stops traffi...",A man sits on bricks pretending to read as stu...,0,https://www.bbc.com/news/world-africa-56362375,"[[224.83755493164062, 96.55413055419922, 402.2...","On DATE, a student protester stops traffic in ...","[['Wednesday', 'DATE'], ['Johannesburg', 'GPE'...",A man sits on bricks pretending to read as stu...,"[['the African National Congress', 'ORG'], ['A...",...,0.444486,"['DATE', 'a student protester', 'traffic', 'GP...","['A man', 'bricks', 'students members', 'ORG',...",[a man sitting on the side of a street with a ...,"[[0.4686562716960907, 0.5873401165008545, 0.27...","[[0.49241191148757935, 0.6341781616210938, 0.1...",0.795884,0.164749,0.477729,True
112,test/112.jpg,"Mr. Trump and Mr. Putin in Helsinki, Finland, ...",Donald Trump attends a working lunch with Vlad...,0,https://www.nytimes.com/2019/01/15/us/politics...,"[[151.3575439453125, 65.49884033203125, 168.84...","Mr. PERSON and Mr. PERSON in GPE, GPE, DATE.","[['Trump', 'PERSON'], ['Putin', 'PERSON'], ['H...",PERSON attends a working lunch with PERSON of ...,"[['Donald Trump', 'PERSON'], ['Vladimir Putin'...",...,0.624944,"['Mr. PERSON', 'Mr. PERSON', 'GPE']","['PERSON', 'a working lunch', 'PERSON', 'GPE',...","[a group of people sitting at a table, an grou...","[[0.7858210206031799, 0.5017732381820679, 0.38...","[[0.7278960943222046, 0.31289157271385193, 0.3...",0.671890,0.227001,0.396821,True
1407,test/1407.jpg,"An electric sign displays ""Save Lives"" in the ...",Amid a patchwork of stay-at-home orders across...,0,https://www.denverpost.com/2020/03/27/denver-s...,"[[101.09113311767578, 204.5328369140625, 145.8...","An electric sign displays ""Save Lives"" in LOC ...","[['the River North Art District', 'LOC'], ['De...",Amid a patchwork of stay-at-home orders across...,"[['Jared Polis', 'PERSON'], ['March 25', 'DATE...",...,0.253211,"['An electric sign', '""WORK_OF_ART', 'LOC', 'G...","['a patchwork', 'home', 'the state', 'various ...","[a bench with a sign on the side of a street, ...","[[0.5199531316757202, 0.3520153760910034, 0.43...","[[0.553579568862915, 0.6885166764259338, 0.255...",0.410825,0.304539,0.156509,True


In [203]:
col = ['img_local_path','caption1','caption2','context_label','caption_3','sbert_wk_score','sbert_wk_score_2','predict','predict_2']

In [205]:
df[(df['predict']==df['predict_2']) & (df['predict']!=df['context_label'])][col]

,img_local_path,caption1,caption2,context_label,caption_3,sbert_wk_score,sbert_wk_score_2,predict,predict_2
14,test/14.jpg,"U.S. President Donald Trump tweeted that, ""Any...",The image claiming ordering a ticket to Trump'...,1,[the man sitting on top of a stage with a podi...,0.554334,0.424993,False,False
20,test/20.jpg,"Martin Fayulu, the opposition leader and runne...",Opposition presidential candidate in the Democ...,0,[a group of people standing in front of each o...,0.463646,0.080946,True,True
21,test/21.jpg,A tribute to Captain Sir Tom Moore lights up P...,A woman plays the violin in Piccadilly Circus ...,0,[woman taking image of a man on a billboard in...,0.420874,0.191306,True,True
28,test/28.jpg,On Tuesday police patrol the school where more...,Police patrol inside the Government Science sc...,0,[a group of men standing next to a baseball gl...,0.445574,0.000000,True,True
30,test/30.jpg,Maine lobsterman catches extremely rare purple...,These are not genuine photographs of a purple ...,1,[two lobsters sitting in the water on top of a...,0.529965,0.324881,False,False
...,...,...,...,...,...,...,...,...,...
1674,test/1674.jpg,Hundreds have reportedly been killed in Tigray...,Hundreds of people walk next to and gather by ...,0,[a group of people standing on a beach with th...,0.496072,0.226165,True,True
1676,test/1676.jpg,"How Julia Jordan, Champion of Female Playwrigh...","“I’m primarily the dog walker, but usually the...",0,"[a woman and a dog on the side of a street, tw...",0.159605,0.213330,True,True
1679,test/1679.jpg,Buildings around the UK were also lit in blue ...,"Falkirk is home to The Kelpies, the largest eq...",0,"[an elephant in blue at night on the street, a...",0.166219,0.274824,True,True
1691,test/1691.jpg,A man exercises on a street in Edinburgh,A man jumps over a skipping rope in a street,0,[a man walking down the street with a skateboa...,0.298860,0.296961,True,True


In [129]:
import spacy
nlp = spacy.load("en_core_web_sm")
def modify_caption_replace_entities(caption_text):
    """
        Utility function to replace named entities in the caption with their corresponding hypernyms
        Args:
            caption_text (str): Original caption with named entities
        Returns:
            caption_modified (str): Modified caption after replacing named entities
    """
    doc = nlp(caption_text)
    caption_modified = caption_text
    caption_entity_list = []
    for ent in doc.ents:
        caption_entity_list.append((ent.text, ent.label_))
        caption_modified = caption_modified.replace(ent.text, ent.label_, 1)
    return caption_modified

In [149]:
df['caption1_modified'] = df['caption1'].apply(lambda x:modify_caption_replace_entities(x))
df['caption2_modified'] = df['caption2'].apply(lambda x:modify_caption_replace_entities(x))

In [ ]:
# False      592  135
# True       258  715
# 0.76882

In [13]:
715 / (135+715)

0.8411764705882353

In [139]:
pd.options.display.max_colwidth = 500
wrong = df[df['context_label']!=df['predict']][['img_local_path','caption1','caption2','context_label','sbert_wk_score','iou','entail']]

In [140]:
print(wrong[wrong['context_label']==True]['entail'].value_counts())
wrong[wrong['context_label']==False]['entail'].value_counts()

neutral          94
contradiction    48
entail            3
Name: entail, dtype: int64


neutral          139
contradiction     68
entail            14
Name: entail, dtype: int64

In [153]:
wrong['i']=wrong['iou']<0.5

In [167]:
wrong[wrong['context_label']==0].sample(5)

,img_local_path,caption1,caption2,context_label,sbert_wk_score,iou,entail,i
1539,test/1539.jpg,The Queen knighted 100-year-old veteran Captain Tom Moore at Windsor Castle on 17 July,The Queen uses a sword to knight Captain Tom Moore,0,0.418063,0.799432,neutral,False
1152,test/1152.jpg,"People tended to hang out near a fire exit, they congregated there, said Mr Roe","Silhouette Club, Hull, 1980s",0,0.043502,0.622934,contradiction,False
147,test/147.jpg,Visitors approach the Meroe Pyramids in northern Sudan on Friday.,A woman walks with children towards the Meroe Pyramids. Nearby a man rides a camel.,0,0.421511,0.599818,neutral,False
882,test/882.jpg,This postal worker was well wrapped up to face the elements in Lauder in the Borders,Lauder under a blanket of snow,0,0.223170,0.700086,contradiction,False
1691,test/1691.jpg,A man exercises on a street in Edinburgh,A man jumps over a skipping rope in a street,0,0.298821,0.876668,neutral,False


In [82]:
v = df['caption2'].str.split(expand=True).stack().value_counts()

In [92]:
stopwords = open('stopwords.txt').read().split('\n')